https://stackoverflow.com/questions/77967527/using-script-optimise-minimize-to-fit-the-sum-of-functions-to-allow-individual-d

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

In [2]:
def func1(x, n, xe, ye):
    b = 1.9992 * n - 0.3271
    return (ye * 1e-12) * np.exp(-b * np.power(x / xe, 1. / n) - 1.)

In [3]:
def func2(x, y0, h):
    return (y0 * 1e-12) * np.exp(-x / h)

In [4]:
def model(x, n, xe, ye, y0, h):
    return (func1(x, n, xe, ye) + func2(x, y0, h))

In [5]:
p = (3.0, 11.2, 5.6, 5.2, 12.1)

In [6]:
np.random.seed(12345)
xexp = np.logspace(-2, np.log10(40), 50, base=10)
#xexp = np.linspace(1e-2, 100, 50)
sigma = 1e-14
sexp = np.full(xexp.shape, sigma)
noise = sexp * np.random.normal(size=xexp.size)
yth = model(xexp, *p) 
yexp = yth + noise

In [7]:
def loss_factory(x, y, s):
    def wrapped(p):
        return 0.5 * np.sum(
            np.power((y - model(x, *p)) / s, 2.) #+ np.log(2 * np.pi * np.power(s, 2.))
        )
    return wrapped

In [8]:
loss = loss_factory(xexp, yexp, sexp)

In [9]:
sol = optimize.minimize(
    loss,
    x0=(10, 10, 5, 5, 10),
    bounds=[
        (1e+0, 2e1),
        (1e-1, 2e1),
        (1e-1, 2e1),
        (1e-1, 2e1),
        (1e-1, 2e1),
    ]
)

In [10]:
sol.x

array([ 3.11220365, 15.57243235,  5.79168983,  5.17425918, 12.09578291])

In [ ]:
xlin = np.linspace()

In [11]:
yhat = model(xlin, *sol.x)
f1hat = func1(xlin, *sol.x[:3])
f2hat = func2(xlin, *sol.x[3:])

NameError: name 'xlin' is not defined

In [ ]:
fig, axe = plt.subplots()
axe.scatter(xexp, yexp, marker='.')
axe.semilogx(xlin, yhat)
axe.plot(xlin, f1hat, "--")
axe.plot(xlin, f2hat, "--")
axe.grid()